### Spotify Login - For One Hour
After running this click Sign-In and login to Spotify. If the Time expires then repeat the process again.

In [ ]:
from ipyauth import ParamsSpotify, Auth

auth = Auth(ParamsSpotify(redirect_uri='http://localhost:8888/callback', client_id="9e4657eefbac41afa98c61f590d8fd51"))
auth

### Common Stuff and Imports


In [ ]:
import requests
from IPython.display import Image
from pandas.io.json import json_normalize
from pandas import DataFrame,read_pickle,merge
from pandas import DataFrame as df
from datetime import datetime,timedelta
import json

import logging
import os

logger = logging.getLogger()
#logger.setLevel(logging.INFO)

def fetch( path, url=None ):
    callPath = url if (url!=None) else ("https://api.spotify.com" + path)
    response = requests.get(callPath , headers= {"Authorization":"Bearer " + auth.access_token })
    if (response.status_code!=200):
        logging.debug("error ")
    return response.json()

def post( path, body, url=None ):
    callPath = url if (url!=None) else ("https://api.spotify.com" + path)
    response = requests.post(callPath, headers= {"Authorization":"Bearer " + auth.access_token }, data=json.dumps(body))
    if (response.status_code!=200):
        logging.debug("error ")
    return response.json()

def delete2( path, body, url=None ):
    callPath = url if (url!=None) else ("https://api.spotify.com" + path)
    response = requests.delete(callPath, headers= {"Authorization":"Bearer " + auth.access_token }, data=json.dumps(body))
    if (response.status_code!=200):
        logging.debug("error ")
    return response.json()


def fetchPage( path, offset, limit):
    res = fetch(  path + "?offset=" + str(offset) + "&limit=" + str(limit))
    return res

def fetchAll( path ):
    more = fetchPage( path, 0, 50)
    limit = more["limit"]
    total = more["total"] - limit
    items = more["items"]
    while((total>0) and (more["next"]!=None )):
        more = fetch( None, url = more["next"] )
        items.extend( more["items"])
        total = total - len(more["items"])
    return items

def fetchPageIds( path, ids): 
    return fetch("{0}?ids={1}".format(path,",".join(ids)))

def fetchAllIds( path, resultField, ids, pageSize=50, existingDf=None):
    if (existingDf is not None):
        keys = existingDf.index.values
        ids = list(filter( lambda id: (id not in keys),ids))
        existingDf = existingDf.reset_index()
        logging.info("Filtering out {0} IDs. Now {1}".format(len(keys), len(ids)))
    total = len(ids)
    logging.info("Requesting {0} rows. {1} ... {2}".format(total, path, resultField))
    offset = 0
    while (offset < total) :
        result = fetchPageIds(path, ids[offset: min(total, offset + pageSize)])
        if (resultField not in result ):
            logger.error("Key not in results: {0}".format( result.keys() ))
            raise Exception("Cannot find key in results")
        items = json_normalize(result[resultField])
        if (existingDf is None):
            logging.info("Creating new DF {0}".format(len(items)))
            existingDf = items
        else:
            existingDf = existingDf.append(items, ignore_index=True )
        offset += len(items)
    return existingDf.to_dict(orient="records")


def addTracks(playlistId, tracks):
    return post("/v1/users/{0}/playlists/{1}/tracks".format(userId,playlistId), {
        "uris": tracks
    })

def createPlaylist(name,description):
    return post("/v1/users/{0}/playlists".format(userId), {
        "name": name,
        "description": description,
        "public": True
    })

def fetchPlaylists():
    return fetchAll("/v1/me/playlists")

def fetchPlaylistTracks( playlistId ):
    return fetchAll("/v1/playlists/{0}/tracks".format(playlistId))


def reconcilePlaylistTracks( playlistName, description, tracks ) :
    existingPlaylists = list(fetchPlaylists())
    existingPlaylist = list(filter(lambda playlist: playlist["name"]==playlistName,existingPlaylists))
    playlist = existingPlaylist[0] if (len(existingPlaylist)>0) else createPlaylist( name = playlistName, description = description )
    existing = json_normalize(fetchPlaylistTracks( playlist["id"] ), sep="_")
    if (existing.empty):
        return addTracks( playlist["id"], tracks )
    existingUris = existing["track_uri"].values.tolist()
    urisDel = list(filter( lambda e: e not in tracks, existingUris))
    urisAdd = list(filter( lambda t: t not in existingUris, tracks))
    while (len(urisDel)>0):
        page = urisDel[-100:]
        bodylist = list(map( lambda x: dict([('uri',x)]), page ))
        url = "/v1/playlists/{0}/tracks".format(str(playlist["id"]))
        delete2(url, { "tracks": bodylist })
        urisDel = urisDel[:-100]
    while (len(urisAdd)>0):
        page = urisAdd[-100:]
        addTracks(playlist["id"],page)
        urisAdd = urisAdd[:-100]
        
user = fetch("/v1/me")
userId = user["id"]
Image(url=user["images"][0]["url"], width=100)


### Read The User Library
This process may take a little while. The library tracks are cached locally, so this step can be skipped.

In [ ]:
data = fetchAll("/v1/me/tracks")
tracksDf = json_normalize(data, sep="_").set_index("track_uri")
tracksDf.to_pickle("mytracks.pkl")
tracksDf.head(2)

### Verify the library cache exists

In [ ]:
tracksDf = read_pickle("mytracks.pkl")
tracksCache = tracksDf[["added_at","track_name","track_album_name","track_album_id"]]
tracksCache.head(2)

### Read the Artists
Load the artists one by one. Use the Pickle File **artists.pkl** as a cache.
* Create a DataFrame using the cache file if one exists (otherwise None)
* Get the list of all unique artist_ids from the previous DF
* Call `fetchAllIds` - using the artists path, the `artists` JSON field path and the cache DF
* Recreate the new artistDf (Dict returned)
* Save the file back to **artists.pkl**

### Create a Track to Artist Table
Pick out the track.artists array for each library track record. The meta (parent record) is the track.id. Use a prefix for both the meta and the record because they both use id.


In [ ]:
artistsPickle = read_pickle("artists.pkl") if (os.path.isfile("artists.pkl")) else None 
artistIds = list(set(artist_and_track["artist_id"].values))
artists = fetchAllIds("/v1/artists","artists",artistIds,existingDf=artistsPickle)
artistsDf = json_normalize(artists).set_index("id")
artistsDf.to_pickle("artists.pkl")

artist_and_track = json_normalize( data=data, record_path=['track','artists'],  meta=[["track","name"],["track","uri"]],  record_prefix='artist_',   sep="_" )
artist_and_track = artist_and_track[['track_name','artist_id','artist_name', 'track_uri']]

artistsDf[["name","genres"]].head(2)


### Read the Albums

In [ ]:
albumsPickle = read_pickle("albums.pkl") if (os.path.isfile("albums.pkl")) else None 
album_ids  = list(set(tracksDf["track_album_id"].values))
albums = fetchAllIds("/v1/albums","albums",album_ids,pageSize=20,existingDf=albumsPickle)
albumsDf = json_normalize(albums, sep="_").set_index("id")
albumsDf.to_pickle("albums.pkl")

albumsDf["released"] = albumsDf.apply(lambda al: datetime.strptime(al["release_date"], "%Y" if (al.release_date_precision=="year") else "%Y-%m" if (al.release_date_precision=="month") else "%Y-%m-%d"), axis=1) 

libraryWithAlbums = merge(tracksDf,albumsDf, left_on="track_album_id", right_index=True, suffixes=("_track","_album"))
libraryWithAlbums[["name","release_date","tracks.total","released"]].head(2)


### Track Features in Library

In [ ]:
featuresPickle = read_pickle("features.pkl") if (os.path.isfile("features.pkl")) else None 
features = fetchAllIds("/v1/audio-features","audio_features",tracksDf["track_id"].values,pageSize=50,existingDf=featuresPickle)
featuresDf = json_normalize(features, sep="_").set_index("uri")
featuresDf.to_pickle("features.pkl")
libraryWithFeatures = merge(libraryWithAlbums,featuresDf, left_index=True, right_index=True, how="outer")
libraryWithFeatures[["track_name","tempo","loudness","energy","released"]].head(2)

---------------------------------

## All Read - let's create an auto playlist



### Running Playlist 
* Tempo between 160 and 200 (for cadence)
* Energy above 0.6
* Danceability above 0.7

In [ ]:
newPlaylist = libraryWithFeatures[ 
#        (libraryWithFeatures.tempo>155) 
#                                  & (libraryWithFeatures.tempo<170) 
#                                  & (libraryWithFeatures.released>(datetime.now()+timedelta(days=-36500))) 
                                  (libraryWithFeatures.energy>0.8 )
                                  & (libraryWithFeatures.loudness>-9 )
                                 & (libraryWithFeatures.danceability>0.8 )]
reconcilePlaylistTracks("Auto Run Fast","Tempo>150 < 190 energy>50",newPlaylist.index.values.tolist())
newPlaylist[["track_name","released","tempo","track_artists","energy","loudness","tempo","danceability"]]
